In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [109]:
df= pd.read_csv('banknote.csv',names=['variance','skewness','curtosis','entropy','class'])
df.head(4)

,variance,skewness,curtosis,entropy,class
0,3.6216,8.6661,-2.8073,-0.44699,0
1,4.5459,8.1674,-2.4586,-1.46210,0
2,3.8660,-2.6383,1.9242,0.10645,0
3,3.4566,9.5228,-4.0112,-3.59440,0


In [110]:
x=df.iloc[:,0:4]
y=df.iloc[:,4:5]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
p=pd.merge(x_train,y_train,on=x_train.index)
p.drop(columns='key_0',inplace=True)

In [111]:
def train(df):
    no_class=df['class'].unique().tolist()
    mean={}
    var={}
    col=df.columns
    class_prob={}
    for i in col[:-1]:
        mean[i]={}
        var[i]={}
        for j in no_class:
            mean[i][j]=df[i][df['class']==j].mean()
            var[i][j]=df[i][df['class']==j].var()
    for i in no_class:
        class_prob[i]=len(df[df['class']==i])/len(df)
    return mean,var,class_prob

In [112]:
mean,var,class_prob=train(p)

In [113]:
##p(height∣female)=(1/(√2*π*variance of female) )*exp^((x − mean height of females)^2 /(2*variance of female height ))
def test(x_test,mean,var,class_prob):
    predict=[]
    col=x_test.columns.tolist()
    for i in range(len(x_test)):
        prob_0=1
        prob_1=1
        for j in col:
            #print(i,j)
            prob_0*=np.exp(-((x_test[j].iloc[i] - mean[j][0])**2) /(2 * var[j][0]))*(1 / np.sqrt(2 * np.pi * var[j][0]))
            prob_1*=np.exp(-((x_test[j].iloc[i] - mean[j][1])**2)/(2 * var[j][1]))* (1 / np.sqrt(2 * np.pi * var[j][1]))
        if prob_0*class_prob[0] < prob_1*class_prob[1]:
            predict.append(1)
        else:
            predict.append(0)
    return predict

In [114]:
result=test(x_test,mean,var,class_prob)

In [115]:
def accuracy(result,y_test):
    count=0
    for i in range(len(result)):
        if result[i]==y_test.iloc[i].values.tolist()[0]:
            count+=1
    return (count/len(y_test))*100

In [116]:
accuracy(result,y_test)

83.25242718446601